## Using the Histogram Toolkit with DL-MONTE

This note assumes you are familiar with the histogram toolkit, and DL-MONTE. See the accompanying notebooks:

1. `intro-ising.ipynb`
2. `dlmonte-util.ipynb`

The histogram toolkit provides a implementation of the Observable
data class for use with DL MONTE.

In [1]:
#Import standard os module for file manipulations
import os
import sys

# You will need to set DL_MONTE_HOME appropriately for
# the local system
MONTEPYTHON_HOME = "/home/jap93/DLMONTE3/dlmontepython_course/"

# Set PYTHONPATH to the 'htk' directory within DL_MONTE_HOME
sys.path.append(MONTEPYTHON_HOME)

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

import dlmontepython.htk.sources.dlmonte as dlmonte


### Example: muVT Ensemble

Following Wilding (PRE 52 692, 1995), we consider simulations performed in the $\mu VT$ (Grand Canonical) ensemble using DL-MONTE. The system under consideration consists of Lennard-Jones particles subject to insertion/deletion moves (only). The Hamiltonian may be written

$$ {\cal H} = -U + \mu N $$

where $U$ is the internal interaction energy, $\mu$ is the chemical potential, and $N$ is the number of particles.

The Lennard-Jones potential is standard

$$ \phi(r) = 4w[(\sigma/r)^{12} - (\sigma/r)^6 ] $$

where $w$ is an energy (the well depth) and sigma is a length scale. In this work, $w$ is taken to be 1 eV, and $\sigma = 1$ Angstrom. The potential is cut off at a radius $r_c$ (here 2.5$\sigma$).

A cubic system of side 4$r_c$ is used in this case ($m = 4$ for Wilding).

The chemical potential is specified by means of the activity $z = \exp(\mu/k_bT)/\Lambda^3$ where $\Lambda$ is a thermal wavelength (see e.g., Allan and Tildesley). Here, the activity is 0.06177 A$^{-3}$.

Mearurements of the total interaction energy

$$ U = \sum_{i,j} \phi (r_{ij}) $$

are collected along with the instantaneous particle number $N$. A non-dimensionalised density may be defined:

$$\rho^\star = N\sigma^3/V$$

along with a non-dimensionalised energy density

$$ u^\star = (4w)^{-1} (\sigma^3/V) U. $$




<br>
<br>
## Running an example simulation



### Reading DL-MONTE data

The DL-MONTE simulation is controlled by three separate input files which must be co-ordinated to produce the correct results.

For a given DL-MONTE run, this is most easily done using the `DLMonteData` object supplied by the toolkit. This will read the relevant input (`CONTROL`, `CONFIG`, and `FIELD` files as outlined above) together with time series of instantaneous measurements from `PTFILE.000`.

All these files are assumed to be available in a single directory or folder. We must also specify the ensemble appropropriate for the simulation (this is not determined automatically at the moment).


In [2]:
import dlmontepython.htk.sources.dlunits
import dlmontepython.htk.histogram
import dlmontepython.htk.util
from dlmontepython.htk.util import Label

data = dlmonte.DLMonteData("/home/jap93/util-dlmonte_workspace")

In [3]:
table = data.to_table()
print(table)

Source:      /home/jap93/util-dlmonte_workspace
Type:        DL-MONTE YAMLDATA Format
Ensemble:    muVT (Grand Canonical)

Parameters: 
Label        Description          Units               Value
volume       Volume               Angstrom^3         1000.0
systemp      Temperature          K                13754.88
zlj          Activity             Volume^-1         0.06177

Observables:
Label        Description          Units            No. Obs.
t            DLM steps            time steps           1000
nmollj       No. molecules lj     None                 1000
energy       Total energy         unknown units         1000


The number of atoms (aka "molecules" here) varies as a function of time:

In [ ]:
data.summary_time_series(plt, "nmollj", 271)

A histogram of the number of particles suggests we are getting some sampling of a low density phase and a high density phase, which is what we expect. (If we were exactly at coexistance, this histogram would be symmetric.)

In [ ]:
data.summary_histogram(plt, "nmollj", 271)


One can check the correlations to make a judgement about how long the simulation needs to be run.

In [ ]:
data.summary_autocorrelation(plt, "energy", 900000)

In [ ]:
data.summary_autocorrelation(plt, "nmollj", 900000)

We  need to get hold of the parameters from the various input files. The volume of the cell is specified in `CONFIG` as a set of unit vectors. In this case the cell size is a 10 Angstrom cube.

The temperature is specifed in the `CONTROL` file with the key `temperature`. The units are Kelvin. The value is 13754.88 K: the tempurature such that $k_BT = 1$eV is 11594.2 K, so the simulation temperature corresponds to about 1.1853 eV (cf epsilon in Lennard Jones of 1 eV). Wilding (1995) has a critical temperature corresponding to $kT \approx 1.1876$ eV.

Energy output (seen above to be in the range of -2000-0) is in units ?

Chemical potential information appears in the `FIELD` file via the parameter "lj" (otherwise referred to as the activity). The value of the activity here is 0.06177 (per A$^3$). The activity is related to the chemical potential via
$z = e^{ \beta \mu} / \Lambda^3$. See Allen and Tildesley, where $\Lambda$ is a thermal length scale.

Here we equate $\Lambda^3$ with the total volume of the system.
Given the volume of the system is 10$^3$ A, $ln(zV) \approx 4.12$ and $\mu \approx 4.8$ eV.

The activity controls the ease with which molecules are created and destroyed in the system: a larger activity favours easier creation of particles, but not destruction of particles. I.e., larger activity favours the higher density phase. 

In [ ]:
activity = data.parameter("zlj")
volume = data.parameter("volume")
temperature = data.parameter("systemp")

print ("Activity    (A^-3)     ", activity)
print ("Volume      (A^3)      ", volume)
print ("Temperature (K)        ", temperature)

k_b = dlmontepython.htk.sources.dlunits.k_boltzmann("ev")
mu = k_b*temperature*numpy.log(volume*activity)
beta = 1.0/(k_b*temperature)

print ("beta            (eV^-1)", beta)
print ("Chemical potential (eV)", mu)

### Using the Histogram class

The aim is to find the chemical potential for which the histrogram
$x = a_M^{-1}(M - <M>)$ is symmetric (and is bimodal following the universal case).

This will be done by reweighting the observations of the number of particles. 



In [ ]:
n = data.observable("nmollj").data
print ("Minimum and maximum number of particles: ", n.min(), n.max())
np = n.copy()
mbar = np.mean()
print ("Average number of particles, and variance: ", mbar, np.var())

In [ ]:
# Get a histogram
nbins = 161
hn = dlmontepython.htk.histogram.Histogram(n, nbins)
m0, m1, m2, m3, minf = hn.statistic()
print ("Moments: ", m0, m1, m2, m3, minf)
hn.plot(plt)
plt.xlabel(r"Density $\rho$", fontsize = 16)
plt.ylabel(r"$P(\rho)$", fontsize = 16)
plt.text(120, 0.0054, "(a) Simulation not quite at coexistance",
         fontsize = 12)
plt.show()

### Reweighting 

Can we reweight the Histogram? Specifically, we want to reweight the density wrt the chemical potential. So, to reweight obsevations of quantity $O$ wrt the number of particles to $\mu'$ we would require

$$ \langle O \rangle_{\mu'} = \frac{\sum_i O_i exp[-\beta(\mu - \mu')N_i]}{\sum_i exp[-\beta(\mu -\mu')N_i]}$$

To reweight a histogram

$$ h_{\mu'}(\rho^\star_j) = (1/N_{obs}) \sum_i \delta(\rho^\star_i, \rho^\star_j) $$

we would require

$$ h_{\mu'}(\rho^\star_j) = \frac{ \sum_i \delta(\rho^\star_i, \rho^\star_j) exp[-\beta(\mu - \mu')N_i]}{\sum_i exp[-\beta(\mu - \mu')N_i]} $$

This can be done via the weights in the denominator.


First, work out the expectation value of $\rho^\star$ at a new chemical potential:


In [ ]:
from dlmontepython.htk.parameter import Parameter

# reduce the chemical potential (favours low density phase)
mu_new = 4.886

# Create a reweighting object for exp(-beta.mu.Ni)
# This takes two Parameter objects, and the relevant Observable
# as arguemnts:
Mu = Parameter(mu, Label("mu", "Chemical potential", "eV"))
Beta = Parameter(-beta, Label("beta", "1/kT", "1/eV"))

r = dlmontepython.htk.histogram.Reweighter("mu", Mu, Beta, data.observable("nmollj"))

print ("Reweighted mean particle number: ", r.reweight_obs(n, mu_new))


Now try the histogram



In [ ]:
# Register the reweighter with the histogtram only once...
hn.add_reweighter(r)

In [ ]:
# Plot a reweighted histogram at mu_new
mu_new = 4.884
hn.weight_wrt("mu", mu_new)
hn.plot(plt)
plt.ylim(ymax=0.01)
plt.xlabel(r"Density $\rho$", fontsize = 16)
plt.ylabel(r"$P(\rho)$", fontsize = 16)
plt.text(120, 0.0054, "(b) Reweight to new chemical potential",
         fontsize = 12)
plt.show()

### Optimisation problem

In [ ]:
def objective(munew, histogram):

    """This objective integrates up to the mean of the histogram
    and measures how close to 1/2 is the result"""
    
    histogram.weight_wrt("mu", munew)
    m0, m1, m2, m3, minf = histogram.statistic()    
    px, xbin = numpy.histogram(hn.obs, bins=hn.nbins, density=True,
                               weights=hn.wgt)
    dx = xbin[1] - xbin[0]
    n = int((m1 - xbin[0])/dx)
    fr = (m1 - xbin[0])/dx - n
    obj = numpy.sum(dx*px[:n]) + fr*dx*px[n+1]
    return numpy.abs(obj - 0.5)
        
# Note Don't be tempted to use the skewness of the distribution
# as the basis for an objective here; single phase has low skewness
# as well as balanced bimodal...

In [ ]:
import scipy.optimize

# brac = (4.88, 4.89) is used to constrain the range of acceptable
# values of mu; it is easy to fall over into a single phase (a Gaussian)
mu_opt = scipy.optimize.brent(objective, (hn,), brack = (4.88, 4.89), \
                              maxiter = 20)

In [ ]:
print ("Best estimate mu_opt", mu_opt)

hn.weight_wrt("mu", mu_opt)

print ("Reweighted moments ", hn.statistic())

We can reweight the original histogram to the predicted co-existance.

In [ ]:
hn.weight_wrt("mu", mu_opt)

hn.plot(plt)
plt.ylim(ymax=0.006)
plt.xlabel(r"Density $\rho$", fontsize = 16)
plt.ylabel(r"$P(\rho)$", fontsize = 16)
plt.text(120, 0.0054, "(c) Reweight to best estimate of coexistance",
         fontsize = 12)
plt.show()